<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

# Load the data

In [0]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname.txt", url) 
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')

In [29]:
vocab = sorted(set(corpus))
print(len(vocab), "chars")

48 chars


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus])

In [0]:
seq_length = 1000
examples_per_epoch = len(corpus)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [33]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>

# Model

In [0]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [35]:
model = keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]))
model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
model.add(tf.keras.layers.Dense(vocab_size))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_5 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense_2 (Dense)              (64, None, 48)            49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [37]:
history = model.fit(dataset, epochs=10)

Epoch 1/10
39/39 [==============================] - 49s 1s/step - loss: 11.8249 - accuracy: 0.1836
Epoch 2/10
39/39 [==============================] - 48s 1s/step - loss: 2.6575 - accuracy: 0.2744
Epoch 3/10
39/39 [==============================] - 49s 1s/step - loss: 2.4001 - accuracy: 0.3081
Epoch 4/10
39/39 [==============================] - 48s 1s/step - loss: 2.3209 - accuracy: 0.3193
Epoch 5/10
39/39 [==============================] - 49s 1s/step - loss: 2.2420 - accuracy: 0.3423
Epoch 6/10
39/39 [==============================] - 49s 1s/step - loss: 2.1698 - accuracy: 0.3636
Epoch 7/10
39/39 [==============================] - 49s 1s/step - loss: 2.1055 - accuracy: 0.3861
Epoch 8/10
39/39 [==============================] - 49s 1s/step - loss: 2.0426 - accuracy: 0.4051
Epoch 9/10
39/39 [==============================] - 48s 1s/step - loss: 1.9834 - accuracy: 0.4246
Epoch 10/10
39/39 [==============================] - 49s 1s/step - loss: 1.9394 - accuracy: 0.4391


In [38]:
main_model = keras.Sequential()
main_model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]))
main_model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.GRU(rnn_units,return_sequences=True,stateful=True,
                              recurrent_initializer='glorot_uniform',
                              recurrent_regularizer=keras.regularizers.l2(0.01)
                              ))
main_model.add(tf.keras.layers.Dense(vocab_size))

main_model.set_weights(model.get_weights())
model = main_model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_3 (Dense)              (1, None, 48)             49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature = 1.0):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [41]:
print(generate_text(model, start_string=u"که ایران چو باغی ست خرم بهار", temperature=0.5))

که ایران چو باغی ست خرم بهار	که گفت کار ز مرد گفت	بیامد به کرد بر آفرین نهان	بدین گوی تو شد به برگند
چو بیم و بین‌از مهر و گوی
به این نه بر آمد به این بند
به روز و بر چنین کند برنگیر
چنین پیش و روز رای نین و رست	بریدند و بر ز تیغان کوه
برآمد بر آمد به خویش به نیز
که بر نیار و دردار به را	ببیدان که شد پندار خون
برو ته روی به مرد بر از خواه	به آن که بود اندرین به جهان
سپه بر شهرگان بر در دریدان بدید	ز بر از بهر و با جهان
که بندیک و بر سرم بر سر از درو بدی
ز تن بی‌آمد و گرد و این ترا بهم	که بین دور بیاران بدید
ز روش و بر آورد به دروی
به را سر که بر دل بی‌من	همه گفت و بر آهی و کرد
چو گفت بر از آید به چند
به گفتند بود نیامد شد	به گنج و روز رو از بدر آفرین
ز کین در چین تو نه در گفت عر
بدین که دید آن رویش به اندر
چنین روی همه را دیر و بی‌راست	به که درز روش آمد روشیار	به نیکی به مران ز درگار
بدین گفت بر نیاد و سیر
به دید بود اندر آن کنوست	که ای بر به کرد شاه و کشت
به زرگی به برد آر و شهریار	سپهد و دیگر میان بدین
چو گفت و بر تن گفت گرداد	بدین گرد آمد بر جنگ نیز	که بی‌روی بر مرد زین روی
بنود بر 